In [18]:
!pip install pypdfium opencv-python pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 19.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pypdfium: filename=pypdfium-0.0.15-py3-none-any.whl size=7838765 sha256=12e25d05597815130639a047ce366252c66d550854f4277eb662e1fb146be9af
  Stored in directory: /root/.cache/pip/wheels/dd/e1/66/ad41b24aac482885e38a262d4af914f5c36bbc72a4fbeaded1
Successfully built pypdfium


In [19]:
!pip install PyPDF4 opencv-python pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 799.6 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for PyPDF4: filename=PyPDF4-1.27.0-py3-none-any.whl size=61226 sha256=dfe983514e05695dce5d37c648204ebca3cbb53a212f984bb1fa7a977f9e3870
  Stored in directory: /root/.cache/pip/wheels/83/cc/14/cb307e5c99235c4497c7895cdb60b4f7ba2a738b6a5fc0d423
Successfully built PyPDF4


In [20]:
!pip install pymupdf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 17.8 MB/s eta 0:00:00


In [21]:
import fitz  # PyMuPDF
import pandas as pd

def extract_text_blocks(pdf_path, page_number):
    pdf_document = fitz.open(pdf_path)
    page = pdf_document.load_page(page_number)
    blocks = page.get_text("dict")['blocks']

    text_blocks = []
    for b in blocks:
        block_text = ''
        if 'lines' in b:
            for l in b['lines']:
                for s in l['spans']:
                    block_text += s['text']
                block_text += '\n'
            x0, _, x1, _ = b['bbox']
            text_blocks.append({'x0': x0, 'text': block_text})
        elif 'text' in b:
            block_text = b['text']
            x0, _, x1, _ = b['bbox']
            text_blocks.append({'x0': x0, 'text': block_text})

    # Sort text blocks based on x-coordinate
    sorted_blocks = sorted(text_blocks, key=lambda x: x['x0'])

    return sorted_blocks

def organize_into_columns(text_blocks):
    columns = []
    current_column = []

    for block in text_blocks:
        if not current_column:
            current_column.append(block)
        else:
            if block['x0'] - current_column[-1]['x0'] < 10:  # Adjust this threshold as per your PDF
                current_column.append(block)
            else:
                columns.append(current_column)
                current_column = [block]

    # Add the last column
    columns.append(current_column)

    return columns

def save_to_excel(columns, output_file):
    max_length = max(len(col) for col in columns)
    df = pd.DataFrame({f"Column_{i+1}": ['' for _ in range(max_length)] for i in range(len(columns))})

    for i, col in enumerate(columns):
        for j, block in enumerate(col):
            df.at[j, f"Column_{i+1}"] = block['text'].strip()

    df.to_excel(output_file, index=False)

if __name__ == "__main__":
    # PDF file path and page number
    pdf_file_path = '/content/keppel-corporation-limited-annual-report-2018.pdf'
    page_no = 11  # Page number - 1 (index starts from 0)

    # Extract text blocks from the specified page
    text_blocks = extract_text_blocks(pdf_file_path, page_no)

    # Organize text blocks into columns
    columns = organize_into_columns(text_blocks)

    # Save columns from page 12 to an Excel file
    output_excel_file = 'output_text_page_12.xlsx'
    save_to_excel(columns, output_excel_file)
